# EBAC - Regressão II - regressão múltipla - Vivian Sant'Anna

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [12]:
import numpy as np
import pandas as pd

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [14]:
filepath = 'https://raw.githubusercontent.com/vfsantanna/exercicios/0c2729e4ca18c27cefaad85e5de300dc90c4240c/previsao_de_renda.csv'
df = pd.read_csv(filepath_or_buffer=filepath)
print(df)


       Unnamed: 0    data_ref  index sexo  posse_de_veiculo  posse_de_imovel  \
0               0  2015-01-01   8682    F             False             True   
1               1  2015-01-01  12830    M              True             True   
2               2  2015-01-01   2885    M              True             True   
3               3  2015-01-01  16168    F              True            False   
4               4  2015-01-01  12438    M             False            False   
...           ...         ...    ...  ...               ...              ...   
14995       14995  2016-03-01   3678    M              True             True   
14996       14996  2016-03-01   3405    M              True             True   
14997       14997  2016-03-01   7944    F              True             True   
14998       14998  2016-03-01  16019    F             False             True   
14999       14999  2016-03-01  12895    F              True             True   

       qtd_filhos        tipo_renda    

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   index                  15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12466 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  mau                    15000 non-null  bool   
 15  re

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

---

#### 1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
- Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
- Mantenha sempre a categoria mais frequente como casela de referência
- Avalie os parâmetros e veja se parecem fazer sentido prático.

In [16]:
# Analisando a frequência das categorias em cada variável:

for coluna in df.columns[3:]:
    print(coluna, '\n', df[coluna].value_counts(), '\n')

sexo 
 sexo
F    10167
M     4833
Name: count, dtype: int64 

posse_de_veiculo 
 posse_de_veiculo
False    9167
True     5833
Name: count, dtype: int64 

posse_de_imovel 
 posse_de_imovel
True     10082
False     4918
Name: count, dtype: int64 

qtd_filhos 
 qtd_filhos
0     10358
1      2992
2      1458
3       165
4        19
14        4
7         2
5         2
Name: count, dtype: int64 

tipo_renda 
 tipo_renda
Assalariado         7826
Empresário          3311
Pensionista         2543
Servidor público    1316
Bolsista               4
Name: count, dtype: int64 

educacao 
 educacao
Secundário             8927
Superior completo      5262
Superior incompleto     599
Primário                193
Pós graduação            19
Name: count, dtype: int64 

estado_civil 
 estado_civil
Casado      10550
Solteiro     1767
União        1168
Separado      838
Viúvo         677
Name: count, dtype: int64 

tipo_residencia 
 tipo_residencia
Casa             13566
Com os pais        637
Governamental  

In [17]:
y, X = patsy.dmatrices(formula_like='''
                                    np.log(renda) ~ sexo 
                                                    + posse_de_veiculo 
                                                    + posse_de_imovel 
                                                    + C(qtd_filhos, Treatment(0))
                                                    + C(tipo_renda, Treatment('Assalariado'))
                                                    + C(educacao, Treatment('Secundário')) 
                                                    + C(estado_civil, Treatment('Casado')) 
                                                    + C(tipo_residencia, Treatment('Casa')) 
                                                    + idade 
                                                    + tempo_emprego 
                                                    + C(qt_pessoas_residencia, Treatment(2))
                                    ''', 
                       data=df)

In [ ]:
sm.OLS(y, X).fit().summary()

#### 2. Remova a variável menos significante e analise:
- Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
- Observe os parâmetros e veja se algum se alterou muito.

In [18]:
# Removendo a variável `tipo_residencia` com o menor P>|t| == 0.200:

smf.ols(formula='''
                np.log(renda) ~ sexo 
                                + posse_de_veiculo 
                                + posse_de_imovel 
                                + C(qtd_filhos, Treatment(0))
                                + C(tipo_renda, Treatment('Assalariado'))
                                + C(educacao, Treatment('Secundário')) 
                                + C(estado_civil, Treatment('Casado')) 
                                + idade 
                                + tempo_emprego 
                                + C(qt_pessoas_residencia, Treatment(2))
                ''', 
        data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.242
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     137.0
Date:                Tue, 27 Feb 2024   Prob (F-statistic):               0.00
Time:                        08:00:20   Log-Likelihood:                -13201.
No. Observations:               12466   AIC:                         2.646e+04
Df Residuals:                   12436   BIC:                         2.668e+04
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       7.2372      0.037    196.640      0.000       7.165       7.309
sexo[T.M]                                                       0.4796      0.014     33.608      0.000       0.452       0.508
posse_de_veiculo[T.True]                                       -0.0018      0.014     -0.131      0.896      -0.029       0.025
posse_de_imovel[T.True]                                         0.0945      0.013      7.122      0.000       0.068       0.120
C(qtd_filhos, Treatment(0))[T.1]                               -0.0352      0.131     -0.269      0.788      -0.291       0.221
C(qtd_filhos, Treatment(0))[T.2]                                0.1021      0.267      0.382      0.703      -0.422       0.626
C(qtd_filhos, Treatment(0))[T.3]                               -0.4917      0.470     -1.045      0.296      -1.414       0.431
C(qtd_filhos, Treatment(0))[T.4]                               -0.1478      0.414     -0.357      0.721      -0.959       0.664
C(qtd_filhos, Treatment(0))[T.5]                               -0.0166      0.398     -0.042      0.967      -0.797       0.764
C(qtd_filhos, Treatment(0))[T.7]                               -0.4736      0.247     -1.916      0.055      -0.958       0.011
C(qtd_filhos, Treatment(0))[T.14]                               0.0664      0.187      0.356      0.722      -0.299       0.432
C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]            -0.0499      0.350     -0.143      0.887      -0.735       0.636
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1790      0.015     12.241      0.000       0.150       0.208
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]          0.3839      0.233      1.646      0.100      -0.073       0.841
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0908      0.021      4.298      0.000       0.049       0.132
C(educacao, Treatment('Secundário'))[T.Primário]               -0.0239      0.071     -0.337      0.736      -0.163       0.115
C(educacao, Treatment('Secundário'))[T.Pós graduação]           0.5550      0.161      3.447      0.001       0.239       0.871
C(educacao, Treatment('Secundário'))[T.Superior completo]       0.0966      0.013      7.181      0.000       0.070       0.123
C(educacao, Treatment('Secundário'))[T.Superior incompleto]    -0.0287      0.031     -0.930      0.353      -0.089       0.032
C(estado_civil, Treatment('Casado'))[T.Separado]               -0.0402      0.131     -0.307      0.759      -0.297       0.216
C(estado_civil, Treatment('Casado'))[T.Solteiro]                0.0176      0.129      0.136      0.891      -0.235       0.270
C(estado_ci

**Resposta:**

Após remover a variável `tipo_residencia`, o modelo não apresentou melhora nem piora significativa. O coeficiente de determinação manteve o mesmo valor de 0.358 e o *p-value* das variáveis teve uma variação muito pequena, praticamente insignificante.

#### 3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor.

In [19]:
smf.ols(formula='''
                np.log(renda) ~ sexo 
                                + posse_de_veiculo 
                                + posse_de_imovel
                                + tipo_renda == 'Empresário'
                                + tipo_renda == 'Servidor público'
                                + educacao == 'Superior completo'
                                + qt_pessoas_residencia == 3
                                + qt_pessoas_residencia == 4
                                + idade 
                                + tempo_emprego
                ''', 
        data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.240
Model:                            OLS   Adj. R-squared:                  0.239
Method:                 Least Squares   F-statistic:                     393.3
Date:                Tue, 27 Feb 2024   Prob (F-statistic):               0.00
Time:                        08:00:41   Log-Likelihood:                -13219.
No. Observations:               12466   AIC:                         2.646e+04
Df Residuals:                   12455   BIC:                         2.654e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                    7.2281      0.034    213.553      0.000       7.162       7.294
sexo[T.M]                                    0.4819      0.014     34.000      0.000       0.454       0.510
posse_de_veiculo[T.True]                     0.0018      0.014      0.133      0.894      -0.025       0.029
posse_de_imovel[T.True]                      0.0965      0.013      7.298      0.000       0.071       0.122
tipo_renda == 'Empresário'[T.True]           0.1792      0.015     12.290      0.000       0.151       0.208
tipo_renda == 'Servidor público'[T.True]     0.0914      0.021      4.324      0.000       0.050       0.133
educacao == 'Superior completo'[T.True]      0.0973      0.013      7.396      0.000       0.072       0.123
qt_pessoas_residencia == 3[T.True]           0.0056      0.017      0.341      0.733      -0.027       0.038
qt_pessoas_residencia == 4[T.True]           0.0572      0.021      2.740      0.006       0.016       0.098
idade                                        0.0053      0.001      7.080      0.000       0.004       0.007
tempo_emprego                                0.0482      0.001     48.464      0.000       0.046       0.050
==============================================================================
Omnibus:                        1.032   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.597   Jarque-Bera (JB):                1.001
Skew:                          -0.012   Prob(JB):                        0.606
Kurtosis:                       3.037   Cond. No.                         241.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Resposta:**


Quando avaliamos o modelo final em relação ao modelo inicial, observamos que não houve mudanças significativas nos indicadores de desempenho. O coeficiente de determinação teve uma leve redução, passando de 0.358 para 0.357, e o Critério de Informação de Akaike (AIC) apresentou uma pequena melhoria, diminuindo de 27190 para 27170.

---